In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
import ee
import geemap
from utils import ee_utils, date_utils, plot_config
from utils.ee_utils import back_to_float

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from typing import Dict, List, Tuple, Union

import warnings

warnings.filterwarnings("ignore")

In [3]:
from filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
)

## Validating ET Blue Maps with Walter Koch's Estimates

This notebook follows a three-step process:

1. **Create Yearly ET Blue Maps (2018-2022):**  
   - Aggregate dekadal ET Blue maps into monthly maps.
   - Filter the maps to include only the growing season months (April through October).
   - Sum the monthly ET Blue maps to produce yearly ET Blue maps.

2. **Calculate Yearly ET Blue Values for Potentially Irrigated Fields:**  
   - Compute ET blue in m3/ha/yr for each crop field for each year.

3. **Compare ET Blue Values with Walter Koch's Estimates:**
   - Make pretty plots

## Aggregate dekadal ET Blue maps into monthly maps

In [4]:
ET_blue_collection = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Zuerich/ET_blue_postprocessed_wapor_10m_ZH_dekadal_2022"
)


ET_blue_collection = ET_blue_collection.map(lambda img: back_to_float(img, 100))

ET_blue_collection.size().getInfo()

36

In [5]:
ee_utils.print_value_ranges(ET_blue_collection, "ET_blue")

Image 1: Min = 0.00, Max = 12.14
Image 2: Min = 0.00, Max = 1.11
Image 3: Min = 0.00, Max = 1.40
Image 4: Min = 0.00, Max = 0.00
Image 5: Min = 0.00, Max = 1.24
Image 6: Min = 0.00, Max = 1.34
Image 7: Min = 0.00, Max = 1.49
Image 8: Min = 0.00, Max = 1.61
Image 9: Min = 0.00, Max = 1.45
Image 10: Min = 0.00, Max = 1.70
Image 11: Min = 0.00, Max = 2.47
Image 12: Min = 0.00, Max = 1.88
Image 13: Min = 0.00, Max = 1.77
Image 14: Min = 0.00, Max = 2.80
Image 15: Min = 0.00, Max = 2.22
Image 16: Min = 0.00, Max = 2.38
Image 17: Min = 0.00, Max = 3.46
Image 18: Min = 0.00, Max = 2.78
Image 19: Min = 0.00, Max = 2.93
Image 20: Min = 0.00, Max = 3.82
Image 21: Min = 0.00, Max = 3.50
Image 22: Min = 0.00, Max = 3.55
Image 23: Min = 0.00, Max = 2.68
Image 24: Min = 0.00, Max = 2.47
Image 25: Min = 0.00, Max = 1.92
Image 26: Min = 0.00, Max = 1.68
Image 27: Min = 0.00, Max = 1.28
Image 28: Min = 0.00, Max = 1.34
Image 29: Min = 0.00, Max = 1.29
Image 30: Min = 0.00, Max = 1.19
Image 31: Min = 0.

In [6]:
ET_blue_collection_monthly = ee_utils.aggregate_to_monthly(
    ET_blue_collection, bands=["ET_blue", "ET_blue_m3"]
)

ET_blue_collection_monthly.size().getInfo()

# ET_blue_collection_monthly = ET_blue_collection


12

## Filter the maps to include only the growing season months (April through October).

In [7]:
ET_blue_collection_monthly_growing_season = ET_blue_collection_monthly.filter(ee.Filter.calendarRange(4, 10, "month"))

date_utils.print_collection_dates(ET_blue_collection_monthly_growing_season)

Dates of images in the collection:
2022-04-01
2022-05-01
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-01


## Sum the monthly ET Blue maps to produce yearly ET Blue maps.

In [8]:
def create_growing_season_yearly_sums(
    collection: ee.ImageCollection,
) -> ee.ImageCollection:
    """
    Create yearly sums from a collection of monthly images for the growing season (April to October).

    Args:
        collection (ee.ImageCollection): Input collection with monthly images for the growing season.

    Returns:
        ee.ImageCollection: Collection with yearly sum images for the growing season.
    """

    def sum_growing_season(year):
        start_date = ee.Date.fromYMD(year, 4, 1)  # April 1st
        end_date = ee.Date.fromYMD(year, 11, 1)  # November 1st (exclusive)
        yearly_sum = collection.filterDate(start_date, end_date).sum()
        return yearly_sum.set("year", year).set(
            "system:time_start", start_date.millis()
        )

    # Get the range of years in the collection
    start_year = ee.Date(collection.first().get("system:time_start")).get("year")
    end_year = ee.Date(
        collection.sort("system:time_start", False).first().get("system:time_start")
    ).get("year")

    # Create a list of years
    years = ee.List.sequence(start_year, end_year)

    # Map over the years to create yearly sums
    yearly_sums = ee.ImageCollection.fromImages(years.map(sum_growing_season))

    return yearly_sums


ET_blue_collection_yearly = create_growing_season_yearly_sums(
    ET_blue_collection_monthly_growing_season
)

In [9]:
ET_blue_collection_yearly.first().bandNames().getInfo()

['ET_blue', 'ET_blue_m3']

## Compute ET blue in m3/ha/yr for each crop field for each year and export

In [10]:
def compute_et_blue_value(
    feature: ee.Feature, et_blue_image: ee.Image, scale: int = 10
) -> ee.Feature:
    """
    Compute ET blue value for a single feature in m3/ha/yr.
    Only computes value if median ET blue within field is > 0.

    Args:
        feature: Input field feature
        et_blue_image: Image with ET blue values in m3
        scale: Scale in meters for computation
    Returns:
        Feature with added ET_blue_m3_ha_yr property
    """
    geometry = feature.geometry()
    area_ha = ee.Number(geometry.area()).divide(10000)

    # First check median ET blue value
    median_et_blue = et_blue_image.reduceRegion(
        reducer=ee.Reducer.median(), geometry=geometry, scale=scale, maxPixels=1e9
    ).get("ET_blue_m3")

    def compute_total_et_blue():
        # If median > 0, calculate total ET blue per hectare
        et_blue_sum = et_blue_image.reduceRegion(
            reducer=ee.Reducer.sum(), geometry=geometry, scale=scale, maxPixels=1e9
        ).get("ET_blue_m3")

        return ee.Number(
            ee.Algorithms.If(
                ee.Algorithms.IsEqual(et_blue_sum, None),
                0,
                ee.Number(et_blue_sum).divide(area_ha).round(),
            )
        )

    # Only compute ET blue if median is > 0
    et_blue_m3_ha_yr = ee.Number(
        ee.Algorithms.If(
            ee.Algorithms.IsEqual(median_et_blue, None),
            0,  # No data case
            ee.Algorithms.If(
                ee.Number(median_et_blue).gt(0),
                compute_total_et_blue(),  # Field is irrigated
                0,  # Field is not irrigated
            ),
        )
    )

    return feature.set("ET_blue_m3_ha_yr", et_blue_m3_ha_yr)


def calculate_et_blue_per_field(
    et_blue_image: ee.Image,
    crop_fields: ee.FeatureCollection,
    scale: Union[int, float] = 10,
) -> Tuple[ee.FeatureCollection, ee.FeatureCollection]:
    """
    Calculate ET_blue in m3/ha/yr for each crop field, setting 0 for excluded and rainfed crops.

    Args:
        et_blue_image: Image containing ET_blue estimates in m3/yr
        crop_fields: Collection of crop field features
        scale: Scale in meters for computations
    Returns:
        Tuple of (all_fields, irrigated_fields) with ET_blue_m3_ha_yr property
    """
    # Get crop type sets
    excluded_crops = get_crops_to_exclude()
    rainfed_crops = get_rainfed_reference_crops()

    # Filter fields based on crop types
    exclude_filter, rainfed_filter = create_crop_filters(excluded_crops, rainfed_crops)
    potentially_irrigated, non_irrigated = filter_crops(
        crop_fields, exclude_filter, rainfed_filter
    )

    # Calculate ET blue for potentially irrigated fields
    irrigated_with_et = potentially_irrigated.map(
        lambda f: compute_et_blue_value(f, et_blue_image, scale)
    )

    # Set ET blue to 0 for non-irrigated fields
    non_irrigated_with_et = non_irrigated.map(lambda f: f.set("ET_blue_m3_ha_yr", 0))

    # Merge collections and filter for actually irrigated fields
    all_fields = irrigated_with_et.merge(non_irrigated_with_et)
    actually_irrigated = all_fields.filter(ee.Filter.gt("ET_blue_m3_ha_yr", 0))

    return all_fields, actually_irrigated

In [11]:
def export_feature_collection(collection: ee.FeatureCollection, task_name: str, asset_id: str): 
    """
    Export the feature collection to an Earth Engine asset.

    Args:
        collection: The feature collection to export.
        year: The year of the feature collection.
        task_name: The name of the export task.
        asset_id: The asset ID to export to.
    """
    task = ee.batch.Export.table.toAsset(
        collection=collection,
        description=task_name,
        assetId=asset_id,
    )
    task.start()

In [12]:
years = range(2022, 2023)

not_irrigated_crops = get_crops_to_exclude()
exclude_filter, _ = create_crop_filters(not_irrigated_crops, not_irrigated_crops)

ET_blue_collection_yearly_list = ET_blue_collection_yearly.toList(
    ET_blue_collection_yearly.size()
)

for year in years:
    index = year - 2022
    nutzung_collection_year = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/ZH_Nutzungsflaechen_2/ZH_Nutzungsflaechen_2022"
    )

    # Create a new property nutzung same as current NUTZUNG
    nutzung_collection_year = nutzung_collection_year.map(
        lambda feature: feature.set("nutzung", feature.get("NUTZUNG"))
    )

    potentially_rainfed, _ = filter_crops(nutzung_collection_year, exclude_filter, _)

    et_blue_image = ee.Image(ET_blue_collection_yearly_list.get(index))

    all_fields, irrigated_fields = calculate_et_blue_per_field(et_blue_image, potentially_rainfed)

    task_name = f"ET_blue_m3_ha_yr_per_field_from_dekadal_wapor_10m_ZH_{year}"
    asset_id = f"projects/thurgau-irrigation/assets/Zuerich/ET_blue_m3_ha_yr_per_field_from_dekadal_wapor_10m_ZH_2022/{task_name}"

    # export_feature_collection(crop_with_et_blue, task_name, asset_id)
    print(f"Exporting {task_name} to {asset_id}")

Exporting ET_blue_m3_ha_yr_per_field_from_dekadal_wapor_10m_ZH_2022 to projects/thurgau-irrigation/assets/Zuerich/ET_blue_m3_ha_yr_per_field_from_dekadal_wapor_10m_ZH_2022/ET_blue_m3_ha_yr_per_field_from_dekadal_wapor_10m_ZH_2022


In [14]:
palette = ["white", "blue", "green", "yellow", "red"]

# Define visualization parameters
visualization = {
    "min": 0,
    "max": 1000,  # Adjust based on your maximum irrigation volume
    "palette": palette,
}

# Convert irrigate_fields_w_estimates to an image for gradient visualization
irrigate_fields = irrigated_fields.reduceToImage(
    properties=["ET_blue_m3_ha_yr"], reducer=ee.Reducer.first()
)

all_fields = all_fields.reduceToImage(
    properties=["ET_blue_m3_ha_yr"], reducer=ee.Reducer.first()
)

# Create a map centered on the area of interest
Map = geemap.Map(center=[47.63915833792603, 8.77542613019931], zoom=12)

# Add the gradient visualization layer to the map
Map.addLayer(irrigate_fields, visualization, "ET_blue_m3_ha_yr")
Map.addLayer(all_fields, visualization, "ET_blue_m3_ha_yr_all_fields")

Map

Map(center=[47.63915833792603, 8.77542613019931], controls=(WidgetControl(options=['position', 'transparent_bg…

In [ ]:
# # Print feature with id =000000000000000048c7 in the nutzung_collection_year
# feature = crop_with_et_blue.filter(ee.Filter.eq("system:index", "000000000000000048c7")).first()
# feature.getInfo()

In [ ]:
# nutzung_collection_year = ee.FeatureCollection(
#     f"projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_{2020}_area"
# )


# potentially_rainfed, _ = filter_crops(nutzung_collection_year, exclude_filter, _)

# et_blue_image = ee.Image(ET_blue_collection_yearly_list.get(2))

# crop_with_et_blue = calculate_et_blue_per_field(et_blue_image, potentially_rainfed)

## Comparing to Walter Koch's estimates

In [ ]:
irrigation_estimate = {
    "Einjährige Freilandgemüse, ohne Konservengemüse": [200, 1000],
    "Kartoffeln": [200, 1000],
    "Freiland-Konservengemüse": [200, 600],
}

IRRIGATION_EFFICIENCY = 0.5

In [ ]:
def filter_dataframe(df: pd.DataFrame, crop_types: List[str]) -> pd.DataFrame:
    """
    Filter the DataFrame to include only the specified crop types.

    Args:
        df (pd.DataFrame): Input DataFrame with 'ET_blue_m3_ha_yr' and 'nutzung' columns.
        crop_types (List[str]): List of crop types to include.

    Returns:
        pd.DataFrame: Filtered DataFrame.
    """
    return df[df["nutzung"].isin(crop_types)]


def plot_histogram_comparison(
    calculated_et_blue: pd.DataFrame,
    farmer_estimates: Dict[str, Tuple[int, int]],
    output_destination: str,
) -> None:
    """
    Create a histogram comparison of calculated ET blue vs farmer estimates.

    Args:
        calculated_et_blue (pd.DataFrame): DataFrame with 'ET_blue_m3_ha_yr' and 'nutzung' columns.
        farmer_estimates (Dict[str, Tuple[int, int]]): Dictionary of farmer estimates for each crop type.
        output_destination (str): Output destination for the plot.
    """
    plot_config.set_plot_style()

    crop_types = list(farmer_estimates.keys())
    num_crops = len(crop_types)

    fig, axs = plt.subplots(1, num_crops, figsize=(6 * num_crops, 6))

    for i, crop in enumerate(crop_types):
        crop_data = calculated_et_blue[calculated_et_blue["nutzung"] == crop][
            "ET_blue_m3_ha_yr"
        ]

        sns.histplot(crop_data, kde=True, ax=axs[i])
        axs[i].axvline(
            farmer_estimates[crop][0],
            color="r",
            linestyle="--",
            label="Farmer's estimate range",
        )
        axs[i].axvline(farmer_estimates[crop][1], color="r", linestyle="--")

        axs[i].set_xlabel("ET Blue (m³/ha/yr)")
        axs[i].set_ylabel("Frequency")
        axs[i].set_title(f"{crop}")

        # Add summary statistics
        mean_value = crop_data.mean()
        median_value = crop_data.median()
        axs[i].axvline(
            mean_value, color="g", linestyle="-", label=f"Average: {mean_value:.0f}"
        )

        axs[i].legend(loc="upper center", bbox_to_anchor=(0.5, -0.15), ncol=2)

    plt.tight_layout()

    # plt.savefig(output_destination, bbox_inches="tight", dpi=300)
    plt.subplots_adjust(bottom=0.2)

In [ ]:
for year in range(2018, 2019):
    ET_blue_per_field = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_m3_ha_yr_per_field_from_dekadal_landsat_30m_2018-2022/ET_blue_m3_ha_yr_per_field_from_dekadal_landsat_30m_{year}"
    )

    double_cropping_image = ee.Image(
        f"projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_{year}"
    )

    ET_blue_per_field = add_double_cropping_info(ET_blue_per_field, double_cropping_image)

    # Filter for single cropping fields
    ET_blue_per_field = ET_blue_per_field.filter(ee.Filter.eq("isDoubleCropped", 0))

    ET_blue_per_field_year_df = geemap.ee_to_df(ET_blue_per_field)
    ET_blue_per_field_year_df = ET_blue_per_field_year_df[
        ["ET_blue_m3_ha_yr", "nutzung"]
    ]

    ET_blue_per_field_year_df["ET_blue_m3_ha_yr"] = (
        ET_blue_per_field_year_df["ET_blue_m3_ha_yr"] / IRRIGATION_EFFICIENCY
    )

    # Filter the DataFrame
    filtered_df = filter_dataframe(
        ET_blue_per_field_year_df, list(irrigation_estimate.keys())
    )

    filtered_df = filtered_df[filtered_df["ET_blue_m3_ha_yr"] > 0]

    # Create the plot
    plot_histogram_comparison(
        filtered_df,
        irrigation_estimate,
        output_destination=f"/Users/cooper/Desktop/Hydrosolutions/ETblue-estimation /images/ET_blue_validation/ET_blue_histogram_comparison_{year}.png",
    )

## Field level estimates for 2022

In [ ]:
PATH_TO_AOI = "projects/thurgau-irrigation/assets/Zuerich/Zuerich_bound"

aoi_feature_collection = ee.FeatureCollection(PATH_TO_AOI)
aoi_geometry = aoi_feature_collection.geometry().simplify(500)

AOI = aoi_geometry.buffer(100)

In [ ]:
fields_w_estimates = ee.FeatureCollection("projects/thurgau-irrigation/assets/ZH_Nutzungsflaechen_2/2022_with_irrigation_estimates")

print(f"Size of fields_w_estimates: {fields_w_estimates.size().getInfo()}")

# Drop fields with PARZNR in this set: UH3685, UH3703, UH3673, UH3674, UH3679, UH3688 and UH3669
fields_w_estimates = fields_w_estimates.filter(
    ee.Filter.inList('PARZNR', ['UH3685', 'UH3703', 'UH3673', 'UH3674', 'UH3679', 'UH3688', 'UH3669']).Not()
)

print(f"Size of fields_w_estimates after dropping: {fields_w_estimates.size().getInfo()}")

modelled_fields = ee.FeatureCollection(
    "projects/thurgau-irrigation/assets/Zuerich/ET_blue_m3_ha_yr_per_field_from_dekadal_wapor_10m_ZH_2022/ET_blue_m3_ha_yr_per_field_from_dekadal_wapor_10m_ZH_2022"
)

parznr_list = fields_w_estimates.aggregate_array('PARZNR').distinct()

# Filter modelled_fields to only keep features with matching PARZNR
filtered_modelled_fields = modelled_fields.filter(
    ee.Filter.inList('PARZNR', parznr_list)
)

print(f"Size of filtered_modelled_fields: {filtered_modelled_fields.size().getInfo()}")

In [ ]:
distinct_use = filtered_modelled_fields.aggregate_array('NUTZUNG').distinct()

distinct_use.getInfo()

In [ ]:
fields_to_check = ee.FeatureCollection(
    filtered_modelled_fields.filter(
        ee.Filter.eq(
            "NUTZUNG", "Landwirtschaftliche Produktion in Gebäuden (z.B. Pilze, Brüsseler)"
        )
    )
)

fields_to_check.size().getInfo()

In [ ]:
IRRIGATION_EFFICIENCY = 0.5

palette = ["white", "blue", "green", "yellow", "red"]

# Define visualization parameters for the images
visualization = {
    "min": 0,
    "max": 1000,
    "palette": palette,
}

# Define visualization parameters for the feature collections - borders only
fields_style = {
    "color": "black",  
    "width": 1,  
    "fillcolor": None,  
}


# Convert fields_w_estimates to an image for gradient visualization
fields_w_estimates_image = fields_w_estimates.reduceToImage(
    properties=["estimated_irrigated_volume"], reducer=ee.Reducer.first()
)

filtered_modelled_fields_image = filtered_modelled_fields.reduceToImage(
    properties=["ET_blue_m3_ha_yr"], reducer=ee.Reducer.first()
)

feature_to_check = ee.FeatureCollection(filtered_modelled_fields.filter(ee.Filter.eq('PARZNR', 'UH3669')))

# Divide by irrigation efficiency to get the actual irrigation volume
filtered_modelled_fields_image = filtered_modelled_fields_image.divide(IRRIGATION_EFFICIENCY)

# Create a map centered on the area of interest
Map = geemap.Map()
Map.center_object(AOI, zoom=12)

# Add the gradient visualization layers to the map
Map.addLayer(fields_w_estimates_image, visualization, "Irrigation Volume (Gradient)")
Map.addLayer(filtered_modelled_fields_image, visualization, "Modelled ET_blue_m3_ha_yr")
Map.addLayer(fields_w_estimates, fields_style, "Fields Borders")

Map.add_colorbar(
    visualization, label="Irrigation Volume (m³/ha)", orientation="horizontal"
)



Map

In [ ]:
# Compute the area of all fields in filtered_modelled_fields where the ET_blue_m3_ha_yr is greater than 0
total_irrigated_area = (
    filtered_modelled_fields.filter(ee.Filter.gt("ET_blue_m3_ha_yr", 0))
    .geometry()
    .area()
    .divide(10000)
    .getInfo()
)
print(f"Total irrigated area: {total_irrigated_area:.2f} ha")

# Print how many hectares are not irrigated
total_non_irrigated_area = (
    filtered_modelled_fields.filter(ee.Filter.eq("ET_blue_m3_ha_yr", 0))
    .geometry()
    .area()
    .divide(10000)
    .getInfo()
)
print(f"Total non-irrigated area: {total_non_irrigated_area:.2f} ha")

# Print area of fields with estimated irrigation volume
total_estimated_irrigated_area = (
    fields_w_estimates.filter(ee.Filter.gt("estimated_irrigated_volume", 0))
    .geometry()
    .area()
    .divide(10000)
    .getInfo()
)
print(f"Total estimated irrigated area: {total_estimated_irrigated_area:.2f} ha")

### Validating field level results for Zuerich

In [ ]:
estimate = ee.FeatureCollection("projects/thurgau-irrigation/assets/ZH_Nutzungsflaechen_2/2022_with_irrigation_estimates")

modelled = ee.FeatureCollection(
    "projects/thurgau-irrigation/assets/Zuerich/ET_blue_m3_ha_yr_per_field_from_dekadal_wapor_10m_ZH_2022/ET_blue_m3_ha_yr_per_field_from_dekadal_2022"
)